In [ ]:
%load_ext autoreload
#%autoreload 2
%matplotlib inline
import sys, os
import matplotlib.pyplot as plt
import dit
import numpy as np

sys.path.append('..')
sys.path.append(os.path.join('..'))

from FACT.helper import *
from FACT.fairness import *
from FACT.data_util import *
from FACT.plot import *
from FACT.lin_opt import *
from sklearn.utils import shuffle
from utils import *
from joblib import Parallel, delayed



In [ ]:
X_train, y_train, X_test, y_test, X_train_removed, X_test_removed, dtypes, dtypes_, sens_idc, race_idx, sex_idx = get_dataset('synth3', corr_sens=True, alpha = 0.95)
plot_synth_data(X_train_removed, y_train, X_train[:,2])


In [ ]:
# split_method=='iid' Scenario 1
# split_method=='heterogeneity' Scenario 2
# split_method=='synergy' Scenario 3


client_data = split_client_data(X_train, y_train, X_test, y_test, X_train_removed, X_test_removed, client_no=2,sex_idx=2, split_method='iid')
plot_synth_data(client_data['client_0']["X_train_removed"], client_data['client_0']["y_train"], client_data['client_0']["X_train"][:,2])
plot_synth_data(client_data['client_1']["X_train_removed"], client_data['client_1']["y_train"], client_data['client_1']["X_train"][:,2])


In [ ]:
clf = sklearn.linear_model.LogisticRegression(solver='lbfgs')
clf.fit(X_train_removed, y_train)
print(clf.score(X_test_removed, y_test))

In [ ]:
gfm, cfm = FairnessMeasures_Fl(X_train, y_train, X_test, y_test, X_train_removed, X_test_removed, client_data, clf)

mats, mats_dict, M_const, b_const = get_fairness_mats_FL(gfm, cfm,  verbose=False)
res = test_fair_instance_FL(gfm, cfm, ['global_DP', 'local_DP'], opt_target='performance', eps=0, err_ub=0.01, lmbd=1, epsdelta=True)
print(res)
measures(res.x)


In [ ]:

xx,yy = np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10)) 

xr = xx.ravel()
yr = yy.ravel()

def calculate_values(i, x, y):
    res = optimization_FL(gfm, cfm, opt_target='performance', eps_g=x, eps_l=y, err_ub=0.1, lmbd=1, epsdelta=True, seed=0)
    meas = measures(res.x)
    glb = meas['I_ZY']
    loc = meas['I_ZY_S']
    z = 1 - res.fun
    return i, z, glb, loc, meas

# Parallel execution of the loop
results = Parallel(n_jobs=-1)(delayed(calculate_values)(i, x, y) for i, (x, y) in enumerate(zip(xr, yr)))
    
# Initialize arrays
z = np.zeros(xr.shape[0])
glb = np.zeros(xr.shape[0])
loc = np.zeros(xr.shape[0])

uni = np.zeros(xr.shape[0])
syn = np.zeros(xr.shape[0])
red =np.zeros(xr.shape[0]) 

# Collect results
for i, z_val, glb_val, loc_val, meas in results:
    z[i] = z_val
    glb[i] = glb_val
    loc[i] = loc_val

    uni[i] = meas['Uniq']
    red[i] = meas['Red']
    syn[i] = meas['Syn']

# Reshape the results
z = z.reshape(xx.shape)
glb = glb.reshape(xx.shape)
loc = loc.reshape(xx.shape)
uni = uni.reshape(xx.shape)
red = red.reshape(xx.shape)
syn = syn.reshape(xx.shape)

z[z > 1.0] = 1.0


fig, axs = plt.subplots()

contour = axs.contourf(xx, yy, z, 20, cmap='viridis')  
plt.colorbar(contour)

axs.set_title('Accuracy Contour Plot')
axs.set_xlabel('Global (glb)')
axs.set_ylabel('Local (loc)')

axs.set_aspect('equal')


In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20, 5))  

# Plot 1
contour = axs[0].contourf(xx, yy, z, 10, cmap=plt.cm.RdBu , alpha=0.9)
fig.colorbar(contour, ax=axs[0])
axs[0].set_title('Accuracy Plot', fontsize=20, fontdict={'fontname': 'Times New Roman'})
axs[0].set_xlabel('Global Disparity $I(Z;\\hat{Y})$', fontsize=20, fontdict={'fontname': 'Times New Roman'})

axs[0].set_ylabel('Local Disparity $\mathrm{I}(Z;\\hat{Y}|S)$', fontsize=20, fontdict={'fontname': 'Times New Roman'})


# Plot 2
contour = axs[1].contourf(xx, yy, uni, cmap='Greens', alpha=1, levels= np.linspace(0,1,20))
fig.colorbar(contour, ax=axs[1])

axs[1].set_title('Unique Disparity', fontsize=20, fontdict={'fontname': 'Times New Roman'})
axs[1].set_xlabel('Global Disparity $I(Z;\\hat{Y})$', fontsize=20, fontdict={'fontname': 'Times New Roman'})
axs[1].set_ylabel('Local Disparity $\mathrm{I}(Z;\\hat{Y}|S)$', fontsize=20, fontdict={'fontname': 'Times New Roman'})


# Plot 3
contour = axs[2].contourf(xx, yy, red, 5, cmap='Greens', alpha=1, levels= np.linspace(0,1,20))
fig.colorbar(contour, ax=axs[2])
axs[2].set_title('Redundant Disparity', fontsize=20, fontdict={'fontname': 'Times New Roman'})
axs[2].set_xlabel('Global Disparity $I(Z;\\hat{Y})$', fontsize=20, fontdict={'fontname': 'Times New Roman'})
axs[2].set_ylabel('Local Disparity $\mathrm{I}(Z;\\hat{Y}|S)$', fontsize=20, fontdict={'fontname': 'Times New Roman'})

# Plot 4
contour = axs[3].contourf(xx, yy, syn, 5, cmap='Greens', alpha=1, levels= np.linspace(0,1,20))
fig.colorbar(contour, ax=axs[3])
axs[3].set_title('Masked Disparity', fontsize=20, fontdict={'fontname': 'Times New Roman'})
axs[3].set_xlabel('Global Disparity $I(Z;\\hat{Y})$', fontsize=20, fontdict={'fontname': 'Times New Roman'})
axs[3].set_ylabel('Local Disparity $\mathrm{I}(Z;\\hat{Y}|S)$', fontsize=20, fontdict={'fontname': 'Times New Roman'})

# Display the plots
plt.tight_layout()
plt.show()
